In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
import missingno as msno


In [ ]:
train=pd.read_csv('../input/mobile-price-classification/train.csv')
test=pd.read_csv('../input/mobile-price-classification/test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

# checking missing value

In [ ]:
#checking missing value
msno.matrix(train)

In [ ]:
train.describe().T

In [ ]:
sns.pairplot(train)

# Effect of battery power on price 

In [ ]:
plt.figure(figsize=[7,7])
sns.boxplot(data=train,x='price_range',y='battery_power')
sns.despine(offset=10, trim=True)
plt.xlabel('Price Class');
plt.title('The battery range in each price class');

As we see above the price class 3 has high battery power , so we can conclude that the more battery power the more increase in the price 

In [ ]:
train['blue'].unique()

In [ ]:
plt.figure(figsize=[7,7])
sns.boxplot(data=train,x='price_range',y='blue')
sns.despine(offset=10, trim=True)
plt.xlabel('Price Class');
plt.title('The battery range in each price class');

Let's see the feature blue effect on price range or not by seeing it's occurance in every price class 

In [ ]:
plt.figure(figsize=[12,5])
sns.countplot(hue='blue',x='price_range',data=train)

Since there are no difference in ocuurance of blue in each price class , we can deduce that it has no effect on pjone price 

Are any of price ranges has high range of clock speed than others ?

In [ ]:
sns.catplot(x="price_range", y="clock_speed", data=train)

oooh ! the range of clock speed is the same in all the 4 price class , so this feature has no effect in our our price 

In [ ]:
train.head()

In [ ]:
plt.figure(figsize=[12,5])
sns.countplot(hue='dual_sim',x='price_range',data=train)
plt.title('number of dual sim in each price range ');

All the 4 price range approximately have the same dual sim range , so may be this feature don't help us in our classification 

In [ ]:
train['fc'].unique()

Are any of price ranges has high value of fc than others ?  

In [ ]:
sns.catplot(x="price_range", y="fc", data=train)
plt.title('range of fc ic each price class');

approximately  the 4 ranges has the same fc , so also this feature has a weak effect in determining the range of the price 

In [ ]:
plt.figure(figsize=[12,5])
sns.countplot(hue='four_g',x='price_range',data=train)
plt.title('number of four_g  in each price range ');

The four price ranges have the advantage of four G   , so i think that these feature has no effect in our classification   

In [ ]:
sns.catplot(x="price_range", y="int_memory", kind="boxen",
            data=train);
plt.title('Range of int Memory in each price range  ');

So as we see, this feature effects in price range since the class 3 has high median and range than the three other ranges , and the class 1 has int memory than 2 and 0 , so we will take this feature in considration in our classification 

In [ ]:
train['m_dep'].unique()

In [ ]:
#display the 
x = sns.boxplot(x="price_range", y="m_dep", data=train)
 

the main of m_dep is the same in all price ranges !! 

In [ ]:
train.corr()['price_range'].T

As we see there are avery weak relationships between the majorty of our feature and our target price ranges 

so we will choose one algorithm to calculate the most important features to use in our classification  

In [ ]:
X=train.drop(['price_range'],axis=1)
y=train['price_range']

In [ ]:
X.shape

In [ ]:
#Import Libraries
from sklearn.feature_selection import SelectFromModel
#----------------------------------------------------

#----------------------------------------------------
#Feature Selection by KBest 
print('Original X Shape is ' , X.shape)


from sklearn.linear_model import LinearRegression
thismodel = LinearRegression()


FeatureSelection = SelectFromModel(estimator = thismodel, max_features = 10) # make sure that thismodel is well-defined
X = FeatureSelection.fit_transform(X, y)

#showing X Dimension 
print('X Shape is ' , X.shape)
print('Selected Features are : ' , FeatureSelection.get_support())

In [ ]:
#Import Libraries
from sklearn.model_selection import train_test_split
#----------------------------------------------------

#----------------------------------------------------
#Splitting data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=44, shuffle =True)

#Splitted Data
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

# VOTING CLASSIFIER 

In [ ]:
#Import Libraries
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
 

#loading models for Voting Classifier
LRModel_ = LogisticRegression(solver='lbfgs', multi_class='multinomial',random_state=33)
GBCModel_ = GradientBoostingClassifier(n_estimators=1000, learning_rate=1.0,max_depth=1, random_state=0)
DTModel_ = DecisionTreeClassifier(criterion = 'entropy',max_depth=3,random_state = 33)
RFModel_ = RandomForestClassifier(n_estimators=100, criterion='gini',max_depth=1, random_state=33)
LDAModel_ = LinearDiscriminantAnalysis(n_components=3 ,solver='svd')

#loading Voting Classifier
VotingClassifierModel = VotingClassifier(estimators=[('LRModel',LRModel_),('GBCModel',GBCModel_),('DTModel',DTModel_),('RFModel',RFModel_),('LDAModel',LDAModel_)], voting='hard')
VotingClassifierModel.fit(X_train, y_train)

#Calculating Details
print('VotingClassifierModel Train Score is : ' , VotingClassifierModel.score(X_train, y_train))
print('VotingClassifierModel Test Score is : ' , VotingClassifierModel.score(X_test, y_test))
print('----------------------------------------------------')

#Calculating Prediction
y_pred = VotingClassifierModel.predict(X_test)
 